### CS 268 - HW4
* Name: Berkay Guler
* Department: NetSys 

# Problem 1)
### A - Augmented Lagrangian:
This method uses a dynamic penalty term that is updated at each iteration to solve for optimization problems with equality constraints only. Given the optimization problem
$$ \min_{x} {f(x)}$$
$$ \text{subject to } h(x) = 0$$

Augmented Lagrangian method employs the penalty function
$$ p(x) = \frac{1}{2} \rho \sum_{i}{(h_i{(x)})^2} - \sum_{i}{\lambda_i h_i(x)}$$
where at each iteration $ \lambda $ is updated as
$$ \lambda^{(k+1)} = \lambda^{(k)} - \rho^{(k)} h(x)$$
and $ \rho $ is updated as
$$ \rho^{(k+1)} = \rho^{(k)} \gamma$$

Minimizing the unconstrained problem $ f(x) + p(x) $ repeteadly with updated $p(x)$ penalty function until convergence is an approximation to minimizing $ f(x) $ in the constrained fashion.

In [1]:
using LinearAlgebra

function augmented_lagrange_method(f, h, x, k_max; ρ=1, γ=2)
    #= Minimizes f subject to h(x) = 0
    source code taken (modified) from: K&W Algorithms for Optimization page 183
    returns:
        x: point where min occurs
        descents: number of gradient descents performed
        convergence_measure: The absolute difference between the last obtained
            minimum value of the objective function f and the one before the last.
    =#
    λ = zeros(length(h(x)))
    descents = 0
    old_f_value = f(x)
    convergence_measure = 0
    for k in 1:k_max
        # penalty function
        p = x -> ρ/2*sum(h(x).^2) - λ⋅h(x)
        # function to optimize
        objective = x -> (f(x) + p(x))
        # steepest gradient descent
        x, descent = gradient_descent(objective, x)
        new_f_value = f(x)
        convergence_measure = abs(new_f_value - old_f_value)
        λ = λ .- ρ*h(x)
        ρ *= γ
        descents += descent
        old_f_value = new_f_value
    end
    return x, descents, convergence_measure
end

augmented_lagrange_method (generic function with 1 method)

### B - Minimization Method:
I am using the steepest descent algorithm with exact line search to solve for the minimization problem occuring in the Augmented Lagrangian Loop. We modify the penalty function at each iteration, leading to gradients of the objective function changing at each iteration. Therefore, I am calculating the gradient of the objective function using numerical differentiation methods built into Julia. One could also exploit the fact that $f(x)$ is fixed. Hence, analythically calculating the gradient of $f(x)$ saves computation. However, $p(x)$ is not fixed and its gradient cannot be calculated with just one hard-coded function.

In [2]:
using ForwardDiff

function gradient_descent(f, x, k_max=100000, α=0.0000001)
    #= performs steepest descent until convergence
    returns:
        x_current: point where min occurs
        k: number of descents taken
    =#
    x_current = x
    k = 1
    while k < k_max
        ∇_x_current = ForwardDiff.gradient(f, x_current)
        d = -∇_x_current / sqrt(dot(∇_x_current, ∇_x_current))
        α = line_search(f, x_current, d)
        x_current = x_current + α*d
        if norm(∇_x_current) < 0.01
            break
        end
        k += 1
    end
    return x_current, k
end


function line_search(f, x, d)
    # source code taken (but modified) from: K&W Algorithms for Optimization page 54
    # returns optimal step size by performing line search of f along d and starting at x
    objective = α -> f(x + α*d)
    # 1-D minimization solver 
    α = minimize(objective)
    return α
end


function minimize(
        func, initial_point=0, 
        initial_step_size=1e-2, convergence_thr=0.000000001;)
    # source code taken (but modified) from: K&W Algorithms for Optimization page 41
    # returns the value at which func attains its min value
    # optimization method used = Golden Section Search 
    # source(s) of code used = K&W Algorithms for Optimization, page 41
    
    a, b = bracket_minimum(func, initial_point, s=initial_step_size, k=2.0)
    ρ = φ - 1
    d = ρ * b  + (1 - ρ)*a
    yd = func(d)
    x_estimate_curr = (a + b)/2
    x_estimate_prev = x_estimate_curr + convergence_thr + 0.0001  
    eval_count = 1
    
    while abs(x_estimate_curr - x_estimate_prev) > convergence_thr
        c = ρ*a + (1 - ρ)*b 
        yc = func(c)
        if yc < yd
            b, d, yd = d, c, yc
        else
            a, b = b, c
        end
        x_estimate_prev = x_estimate_curr
        x_estimate_curr = (a + b)/2
        eval_count += 1
    end
    # return x_estimate_curr, abs(x_estimate_curr - x_estimate_prev), eval_count
    return x_estimate_curr
end


function bracket_minimum(fnc, x=0; s=1e-4, k=2.0)
    # returns an interval in which fnc has a minimum
    # source(s) of code used = K&W Algorithms for Optimization, page 36
    a, ya = x, fnc(x)
    b, yb = a + s, fnc(a + s) 
    if yb > ya
        a, b = b, a
        ya, yb = yb, ya
        s = -s
    end
    while true
        c, yc = b + s, fnc(b + s) 
        if yc > yb
            return a < c ? (a, c) : (c, a)
        end
        a, ya, b, yb = b, yb, c, yc
        s *= k
    end
end

bracket_minimum (generic function with 2 methods)

### C - Test Functions:
I am using the test functions taken from my HW2 submission. I am using the same constraint of restricting the sum of squares of all entries to be equal to some non-negative constant $k$.

1) 10-D Booth's Function:
$$ f(x) = \sum_{i=1}^{9} {[(x_i + 2x_{i+1}-7)^2 + (2x_i + x_{i+1}-5)^2]}$$

2) 10-D Rosenbrock Function:
$$ f(x) = \sum_{i=1}^{9} {[(a-x_i)^2+b(x_{i+1}-x_i^2)^2]}$$

Hence the final optimization objective looks like
$$ \min_{x} {f(x)}$$
$$ \text{subject to } h(x) = 0$$
$$ \text{where } h(x) = (\sum_{i} {x_i^2}) - k$$

In [3]:
using Base.MathConstants

# 10-D Rosenbrock Test Function
function rosenbrock_10D(x, a=1, b=1)
    summation = 0
    for i in 1:9
        summation += ((a - x[i])^2 + b*(x[i+1]-x[i]^2)^2)
    end
    return summation
end

# quadractic function
function booths_10D(x)
    summation = 0
    for i in 1:9
        summation += ((x[i] + 2*x[i+1] - 7)^2 + (2*x[i] + x[i+1] - 5)^2)
    end
    return summation
end

function constraint(x, k=5)
    # returns the value of constraint h
    summation = 0
    for i in 1:length(x)
        summation += (x[i])^2
    end
    return summation - k
end

function sample_exp_uniform_ndim(num_samples=100, ndim=10, range=7)
    # returns 10-D test points
    columns = []
    for i in 1:ndim
        # sample num_samples numbers from [-range, range]
        x = range*(rand(num_samples)) .* rand((-1, 1), num_samples)
        # raise to the power of e and select a random sign
        y = e.^x .* rand((-1, 1), 100)
        append!(columns, [y])
    end
    return hcat(columns...)
end

sample_exp_uniform_ndim (generic function with 4 methods)

### D - Results:
The following metrics are used to evaluate the performance on both test functions. For each metric, mean and standard deviation over 100 test points are reported. Each test point $x^{(i)}=(x_1, x_2, ..., x_{10})$ is selected as follows.
$$Y_i∼Uniform(-a,a)$$ $$X^{(i)} = se^{Y_i}$$ $$ \text{ where } s∼Uniform(\{-1, 1\})$$

* **Absolute error in constraint:** The distance of $h(x)$ from 0. Measures how well the solution satisfies the constraint.
* **Elapsed Time:** Elapsed time in miliseconds.
* **Last Value of Convergence Measure:** The absolute difference between the last obtained minimum value of the objective function $f$ and the one before the last.
* **Number of Descents:** Sum of number of total descents performed within the gradient descent loops.

**Note:** I do not report the absolute error in $f$ because I could not calculate the analytical solution.

In [4]:
# max iteration for augmented lagrangian
max_iteration = 20
function_names = ["Booth's", "Rosenbrock"] 
functions = [booths_10D, rosenbrock_10D]  
# store results in a dict
database = Dict()

test_points = sample_exp_uniform_ndim()

for (function_, f) in zip(function_names, functions)
    if !(function_ in keys(database))
        database[function_] = Dict("error" => [], "time" => [], "conv" => [], "descent" => [])
    end
        
    for i in 1:size(test_points)[1]
        test_point = test_points[i, :]
        @assert length(test_point) == 10
        
        tic = time()
        x, num_descents, convergence_measure = augmented_lagrange_method(
            f, constraint, test_point, max_iteration)
        # 1000 to convert to ms
        elapsed_time = (time() - tic) * 1000

        push!(database[function_]["error"], abs(constraint(x)))
        push!(database[function_]["time"], elapsed_time)
        push!(database[function_]["descent"], num_descents)
        push!(database[function_]["conv"], convergence_measure)
    end
end

In [5]:
using Statistics 

# get means for each columns
mean_error_col = [mean(database[f_name]["error"]) for f_name in keys(database)]
mean_time_col = [mean(database[f_name]["time"]) for f_name in keys(database)]
mean_descent_col = [mean(database[f_name]["descent"]) for f_name in keys(database)]
mean_conv_col = [mean(database[f_name]["conv"]) for f_name in keys(database)]

# get stds for each columns
std_error_col = [std(database[f_name]["error"]) for f_name in keys(database)]
std_time_col = [std(database[f_name]["time"]) for f_name in keys(database)]
std_descent_col = [std(database[f_name]["descent"]) for f_name in keys(database)]
std_conv_col = [std(database[f_name]["conv"]) for f_name in keys(database)]

2-element Vector{Float64}:
 1.0285860192888929e-7
 1.7259725429359137e-9

### D.1 - Mean Table:

In [6]:
using TypedTables

mean_table = Table(
    Func = ["10-D Booth's Function", "10-D Rosenbrock"], 
    Error = mean_error_col,
    Convergence = mean_conv_col, NumDescents = mean_descent_col,
    Time = mean_time_col)

Table with 5 columns and 2 rows:
     Func                   Error       Convergence  NumDescents  Time
   ┌─────────────────────────────────────────────────────────────────────
 1 │ 10-D Booth's Function  2.61456e-9  1.25344e-7   68.06        8.1879
 2 │ 10-D Rosenbrock        2.29206e-9  1.43021e-9   75.92        2.93625

### D.1 - Std Table:

In [7]:
std_table = Table(
    Func = ["10-D Booth's Function", "10-D Rosenbrock"], 
    Error = std_error_col,
    Convergence = std_conv_col, NumDescents = std_descent_col,
    Time = std_time_col)

Table with 5 columns and 2 rows:
     Func                   Error       Convergence  NumDescents  Time
   ┌─────────────────────────────────────────────────────────────────────
 1 │ 10-D Booth's Function  1.1748e-9   1.02859e-7   1.72808      75.3007
 2 │ 10-D Rosenbrock        1.42746e-9  1.72597e-9   5.29852      23.4223

# Problem 2)
### A - Simulated Annealing:
Simulated annealing method samples a datapoint from a probability distribution conditioned on the current point. The sampled point is always accepted if it leads to a descent. If the sampled point leads to an ascent the new point is accepted with some probability which is a function of the temperature. 

To obtain the new datapoint $\tilde{x}$, I am using a Gaussian distribution with mean $x$, the current point, and with unit diagonal covariance matrix.
$$ P(\tilde{x}|x)= N(\tilde{x};x,\sigma^2 I)$$
Another way to view this is adding zero mean Gaussian noise to the current point $x$ to obtain $\tilde{x}$
$$ \tilde{x} = x + \sigma \epsilon \text{  where } \epsilon \sim N(0, I)$$ 
The obtained point is discarded or accepted based on the following decision rule. 
$$
\begin{cases} 
\text{always accept } & \text{if } \Delta y \le 0 \\
\text{accept with probability }e^{-\Delta y/t} & \text{otherwise}
\end{cases}
$$

where $ \Delta y = f(\tilde{x}) - f(x$) and $t$ follows the fast annealing schedule. 
$$ t^{(k+1)} = t^{(k)} \frac{k}{k+1} $$ 

In practice, initial temperature, decay schedule, and variance of the noise greatly affects the performance of the algorithm. I tested several different parameters to minimize the absolute error.  

In [8]:
using Random

function fast_annealing_scheduler(t, k)
    # returns the next temperature given the current temperature
    return t * (k/(k+1))
end


function simulated_annealing(f, x, t_0, scheduler, k_max; verbose=false, report_every_k=1000, σ=1)
    #= performs fast simulated annealing with gaussian noise
    returns:
        x_best: the best point we come across during the search 
        num_ascents: number of ascents performed
        num_descents: number of descents performed
    =#
    y = f(x)
    x_best, y_best = x, y
    m = length(x)
    t = t_0
    num_descents = 0
    num_ascents = 0 
    for k in 1 : k_max
        x′ = x + σ * randn(m)
        y′ = f(x′)
        Δy = y′ - y
        if Δy ≤ 0
            x, y = x′, y′
            num_descents += 1
        elseif rand() < exp(-Δy/t)
            x, y = x′, y′
            num_ascents += 1
        end
        
        if y′ < y_best
            x_best, y_best = x′, y′
        end
        
        if verbose && k % report_every_k == 0
            println("\nStep $k")
            println(x_best)
            println("Temp: $t, prob: $(exp(-Δy/t))\n")
        end
        t = scheduler(t, k)
    end
    return x_best, num_ascents, num_descents
end

simulated_annealing (generic function with 1 method)

### B - Test Functions:
I am using the test functions taken from my HW2 submission. 

1) 10-D Booth's Function:
$$ f(x) = \sum_{i=1}^{9} {[(x_i + 2x_{i+1}-7)^2 + (2x_i + x_{i+1}-5)^2]}$$

2) 10-D Rosenbrock Function:
$$ f(x) = \sum_{i=1}^{9} {[(a-x_i)^2+b(x_{i+1}-x_i^2)^2]}$$

**Note 1:** For $a=1$, Rosenbrock function takes its minimum value
$f(x^*) = 0$ at $x ^* = \{x_i = 1| i=1,2,...,10\}$

**Note 2:** Because the minimum of Booth's function is not easy to obtain analythically, I am using Julia optim library to report an approximate absolute error.

### C - Results:
The following metrics are used to evaluate the performance on both test functions. For each metric, mean and standard deviation over 100 test points are reported. Each test point $x^{(i)}=(x_1, x_2, ..., x_{10})$ is selected as follows.
$$Y_i∼Uniform(-a,a)$$ $$X^{(i)} = se^{Y_i}$$ $$ \text{ where } s∼Uniform(\{-1, 1\})$$
* **Absolute error:** Distance between the ground truth minimum value of function $f$ and the minimum found by the algorithm.
* **Elapsed Time:** Elapsed time in miliseconds.
* **Number of Ascents:** Number of total ascents performed. Ascending generally occurs at higher temperatures.
* **Number of Descents:** Number of total descents performed.

In [9]:
using Optim

function_names = ["Booth's", "Rosenbrock"] 
functions = [booths_10D, rosenbrock_10D]
database = Dict()
# sample test pisoints
test_points = sample_exp_uniform_ndim()
# ground truth min values
bt_res = optimize(booths_10D, test_points[1,:], LBFGS())
bt_gt_min_x = Optim.minimizer(bt_res)
bt_gt_min_f = booths_10D(bt_gt_min_x)
rb_gt_min_x = [1 for i in 1:10]
rb_gt_min_f = rosenbrock_10D(rb_gt_min_x)

zip_iter = zip(function_names, functions, (bt_gt_min_f, rb_gt_min_f))

# simulated annealing parameters
initial_temp = 1000
total_steps = 10000000
report_freq = 100000
sigma = 0.1
verbose = false

for (function_, f, min_f) in zip_iter
    if !(function_ in keys(database))
        database[function_] = Dict("error" => [], "time" => [], "ascent" => [], "descent" => [])
    end
        
    for i in 1:size(test_points)[1]
        test_point = test_points[i, :]
        @assert length(test_point) == 10
        
        tic = time()
        x, num_asc, num_desc = simulated_annealing(
            f, test_point, initial_temp,
            fast_annealing_scheduler, total_steps,
            report_every_k=report_freq,
            verbose=verbose, σ=sigma)
        elapsed_time = (time() - tic) * 1000
        abs_error = abs(min_f - f(x))
        
        push!(database[function_]["error"], abs_error)
        push!(database[function_]["time"], elapsed_time)
        push!(database[function_]["descent"], num_desc)
        push!(database[function_]["ascent"], num_asc)
    end
end

In [10]:
mean_error_col = [mean(database[f_name]["error"]) for f_name in keys(database)]
mean_time_col = [mean(database[f_name]["time"]) for f_name in keys(database)]
mean_descent_col = [mean(database[f_name]["descent"]) for f_name in keys(database)]
mean_ascent_col = [mean(database[f_name]["ascent"]) for f_name in keys(database)]

std_error_col = [std(database[f_name]["error"]) for f_name in keys(database)]
std_time_col = [std(database[f_name]["time"]) for f_name in keys(database)]
std_descent_col = [std(database[f_name]["descent"]) for f_name in keys(database)]
std_ascent_col = [std(database[f_name]["ascent"]) for f_name in keys(database)]

2-element Vector{Float64}:
 345.0588215116813
 637.9315421142323

### C.1 - Mean Table:

In [11]:
mean_table = Table(
    Func = ["10-D Booth's Function", "10-D Rosenbrock"], 
    Error = mean_error_col,
    NumAscents = mean_ascent_col, NumDescents = mean_descent_col,
    Time = mean_time_col)

Table with 5 columns and 2 rows:
     Func                   Error       NumAscents  NumDescents  Time
   ┌────────────────────────────────────────────────────────────────────
 1 │ 10-D Booth's Function  0.0136245   329.84      8296.69      1377.92
 2 │ 10-D Rosenbrock        0.00850584  753.71      8456.42      1195.96

### C.1 - Std Table:

In [12]:
std_table = Table(
    Func = ["10-D Booth's Function", "10-D Rosenbrock"], 
    Error = std_error_col,
    Convergence = std_conv_col, NumDescents = std_descent_col,
    Time = std_time_col)

Table with 5 columns and 2 rows:
     Func                   Error       Convergence  NumDescents  Time
   ┌─────────────────────────────────────────────────────────────────────
 1 │ 10-D Booth's Function  0.00279292  1.02859e-7   5215.73      47.4244
 2 │ 10-D Rosenbrock        0.00187578  1.72597e-9   5045.88      38.1558